In [ ]:
# загружаем файл heart.csv


In [ ]:
# устанавливаем нужную версию библиотеки sklearn, чтобы ответы сошлись
!pip install scikit-learn==0.22.1

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


1. Какой процент пациентов из представленных в данных имеет заболевание сердца (`'target' = 1`)? Ответ округлите до двух знаков после запятой.

In [ ]:
all = data.shape[0]
notall = data[data['target'] == 1]
notall = notall.shape[0]
print(round(notall/all,2))





0.54


Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.33, random_state=13)
X_train.shape, X_test.shape

((203, 13), (100, 13))

2. Обучите линейный классификатор из `sklearn` (`sklearn.linear_model.SGDClassifier`) с максимальным числом итераций `max_iter=1000`, постоянным значением шага градиентного спуска (`learning_rate='constant'`), равным `eta0=0.1`. В качестве сида поставьте `random_state=13`. Параметр регуляризации отключите: `alpha=0`. Класс `sklearn.linear_model.SGDClassifier` объединяет в себе разные линейные модели - чтобы получить логистическую регрессию, зафиксируйте параметр `loss='log'`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html).

   Какое значение свободного коэффициента у полученного линейного классификатора? Ответ округлите до двух знаков после запятой.

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(max_iter=1000, learning_rate='constant', eta0=0.1, random_state=13,alpha=0,loss='log')
clf.fit(X_train,y_train)

SGDClassifier(alpha=0, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=0.1, fit_intercept=True, l1_ratio=0.15,
              learning_rate='constant', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=13, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [ ]:
round(clf.intercept_[0],2)

-0.85

3. Какое значение L2-нормы вектора весов (без учета свободного коэффициента) у полученного линейного классификатора? Ответ округлите до двух знаков после запятой.

_Напоминание. L2-норма вектора $v = (v_1, \ldots, v_n)$ - это корень из суммы квадратов его элементов:_

$$
\|v\|_2 = \sqrt{\sum\limits_{i=1}^nv_i^2}
$$

In [ ]:
import math
sum = 0
L2norm = 0
for item in clf.coef_:
  sum = sum + item*item
for item in sum:
  L2norm = L2norm + item
print(round(math.sqrt(L2norm),2))

175.98


4. Найдите долю правильных ответов классификатора на тестовой части выборки **(в процентах)**. Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [ ]:
round(clf.score(X_train,y_train)*100,2)



52.71

5. В задаче классификации, как и в задаче регрессии, для оптимизации линейных моделей можно применять регуляризацию. Этот метод реализован и в `sklearn.linear_model.SGDClassifier` - параметр регуляризации обозначается параметром `alpha`. За тип регуляризации (L1, L2 или обе сразу) отвечает параметр `penalty`. 

   Обучите классификатор заново с параметром регуляризации `alpha=0.1` и типом регуляризации `penalty='l1'`. Оставьте максимальное число итераций, равное `max_iter=1000` и сид `random_state=13`. Также вместо постоянного значения шага градиентного спуска используйте оптимальное (`learning_rate='optimal'`), которое, кстати, зависит от `alpha` (о том, как именно он вычисляется и какие еще параметры можно выбрать, можно подробнее прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)). В данном случае значение начального шага градиентного спуска `eta0` никак не участвует в обучении.
   
   Отличается ли качество полученного классификатора от качества первого? Какая доля правильных ответов получается теперь на тестовой выборке? Выразите ее **в процентах**, ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [ ]:
clf = SGDClassifier(max_iter=1000, learning_rate='optimal', eta0=0.1, random_state=13,alpha=0.1,loss='log',penalty='l1')
clf.fit(X_train,y_train)
round(clf.score(X_train,y_train)*100,2)



68.47

6. Найдите L2-норму вектора весов для полученного классификатора (заметьте, как на нее повлияла регуляризация). Ответ округлите до двух знаков после запятой.

    Заметьте, что вектор стал более разреженным, и в нем появились нулевые элементы - это результат действия L1-регуляризации.

In [ ]:
sum = 0
L2norm = 0
for item in clf.coef_:
  sum = sum + item*item
for item in sum:
  L2norm = L2norm + item
print(round(math.sqrt(L2norm),2))


19.91


7. Проверьте, как полученные классификаторы предсказывают не классы, а вероятности классов - так как мы работаем с логистической регрессией, это можно сделать. Посмотрите на вероятности, которые выдает первый классификатор (обученный с постоянным шагом градиентного спуска и без регуляризации) на тестовой части выборки. В этом вам поможет метод `predict_proba`. Результатом его работы будет список размера $N\times 2$, где $N$ - это число объектов. В каждом столбце списка находятся вероятности соответствующего класса для объектов. Поэтому если вам нужен положительный класс, вас интересует последний столбец.

    Какое получается значение AUC-ROC? Ответ округлите до двух знаков после запятой.

In [ ]:
clf = SGDClassifier(max_iter=1000, learning_rate='constant', eta0=0.1, random_state=13,alpha=0,loss='log')
clf.fit(X_train,y_train)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1])

0.5

8. Посмотрите на вероятности, которые выдает второй классификатор (обученный с оптимальным шагом градиентного спуска и с регуляризацией) на тестовой части выборки. Как отличаются эти вероятности от вероятностей первого классификатора?

   Посчитайте значение AUC-ROC второго классификатора. Ответ округлите до двух знаков после запятой.

In [ ]:
clf = SGDClassifier(max_iter=1000, learning_rate='optimal', eta0=0.1, random_state=13,alpha=0.1,loss='log',penalty='l1')
clf.fit(X_train,y_train)
from sklearn.metrics import roc_auc_score
print(round(roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1]),2))



0.71


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


9. Какой признак является самым важным по мнению лучшей модели (имеет наибольший по модулю коэффициент) для принятия решения?

In [ ]:
for item in clf.coef_[0]:
  print(abs(item))



18.749089644173225
0.0
3.56317522758753
2.4707517614980996
0.7202683397833803
0.0
0.0
3.939926656694808
0.0
0.6735934834788498
0.0
3.096234035230421
0.0


**Первый признак - age**